In [38]:
import requests
import pandas as pd
import json

In [2]:
token = '5e46444db46f4bb59b1bd6b077385b7a86c5d401a43ca904826186b4ee8de8c6'

In [5]:
resp = requests.get('https://api.esios.ree.es/indicators', 
                    headers=´{'Authorization': 'Token token=%s' % (token_personal)})

SyntaxError: invalid character in identifier (<ipython-input-5-d2c4810492e6>, line 2)

In [6]:
requests.get?

In [10]:
pd.get_option("display.max_columns")

20

In [13]:
pd.options.display.max_columns

20

In [28]:
def API_Esios(fecha_ini,fecha_fin,ind):

    Headers={'Accept': 'application/json; application/vnd.esios-api-v1+json',
                  'Content-Type': 'application/json',
                  'Host': 'api.esios.ree.es',
                  'Authorization': 'Token token="5e46444db46f4bb59b1bd6b077385b7a86c5d401a43ca904826186b4ee8de8c6"',
                  'Cookie':''}

    Params = {'start_date': fecha_ini, 'end_date': fecha_fin}

    URL = 'https://api.esios.ree.es/indicators/' + str(ind)
    
    r = requests.get(URL,headers=Headers,params=Params)
    
    assert r.status_code==200
    
    data=json.loads(r.content)
    
    df_data=pd.DataFrame(data['indicator']['values'])
    
    df=pd.DataFrame()        
    df['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data['datetime']],yearfirst=True,format='%Y-%m-%d %H:%M:%S') 
    df['value']=df_data['value']
    if (ind==475) | (ind==476):
        df=df.groupby('datetime')['value'].sum()
        df=pd.DataFrame(df)
    else:
        df=df.set_index('datetime')

    return df

In [29]:
fecha_ini_='2015-01-01 00:00:00'   
fecha_fin_='2018-12-31 23:59:29'

In [35]:
chunks=[['2015-01-01 00:00:00','2016-09-30 23:59:59'],
        ['2016-10-01 00:00:00','2017-09-30 23:59:59'],
        ['2017-10-01 00:00:00','2018-12-31 23:59:59']]

In [47]:
d={1209: 'Demanda_20',
   1210: 'Demanda 21',
   1211: 'Demanda 2A'}

In [48]:
datos_esios=pd.DataFrame()  #será nuestro set de datos
for i,j in chunks:
    datos_esios_sub=pd.DataFrame()
    for k,v in d.items():
        if (k==600) | (k==612) | (k==613):
            historico=solicita_datos_precios(i,j,k,v)
        else:    
            historico=API_Esios(i,j,k)
            historico.columns=[str(v)]
        datos_esios_sub=pd.concat([datos_esios_sub,historico],axis=1)
    datos_esios=pd.concat([datos_esios,datos_esios_sub])

KeyError: 'datetime'